In [ ]:
import pandas as pd
import numpy as np
import os

# 设定路径
rpath = 'D:\\maran10\\Documents\\日常工作\\商机表\\6.29商机反馈\\'
wpath = 'D:\\maran10\\Documents\\日常工作\\商机表\\'
sep_path = 'D:\\maran10\\Documents\\日常工作\\商机表\\各区域拆分\\'
files = os.listdir(rpath)

opp = pd.read_excel(wpath+'全国商用线下商机表6.18.xlsx',encoding = 'gbk')
# 下面这里是处理第一行的注释
opp.columns = opp.iloc[0]  
opp = opp.loc[1:,:]

opp  # 看一下原表格


In [ ]:
# 循环读各区域提报的新商机表
count=0  # 计循环次数
for f in files: 
    if not ".xlsx" in f:
        continue
    opp_region = pd.read_excel(rpath+f,encoding = "gbk")
    opp_region.columns = opp_region.iloc[0]
    opp_region = opp_region.loc[1:,:]
    count = count+1
#     print(opp_region.head(5))
    print("第几次循环：",count)
    print(f,"该文件中的数据条数",len(opp_region))
    
    # 直接筛选更新了的，没更新的不用管

    opp_new = opp_region[opp_region['本周是否更新']=="是"]
    print(f,"更新数据条数",len(opp_new))
    
    if count == 1:  # 第一次要先生成空表才能concat
        opp_renew = pd.DataFrame(columns = list(opp_new))  # 项目唯一编号不为空的，是更新商机
        opp_totalnew = pd.DataFrame(columns = list(opp_new))  # 项目唯一编号为空的，是新增商机
    
    opp_renew = pd.concat([opp_renew,opp_new[pd.notnull(opp_new['项目唯一编号\n（总部填写）'])]].copy())
    opp_totalnew = pd.concat([opp_totalnew,opp_new[pd.isnull(opp_new['项目唯一编号\n（总部填写）'])]].copy())
    
    print("更新的项目条数",len(opp_renew))
    print("新增的项目条数",len(opp_totalnew))
    
#### 写出表格
opp_renew.to_excel(wpath+'全国更新的商机项目.xlsx',index=False,encoding='gbk')
opp_totalnew.to_excel(wpath+'全国新增的商机项目.xlsx',index=False,encoding='gbk')

# 接下来要对新增的商机项目进行手动编号，编号后再导入

In [ ]:
a = opp['项目唯一编号\n（总部填写）'].value_counts()
print("项目唯一编号重复量",len(a[a>1]))#这里应该为0
# 这里的去重只是为了保险，完全可以不要这一步
opp_dp = opp.drop_duplicates(subset=['项目唯一编号\n（总部填写）'],keep='first', inplace=False)
print(len(opp_dp))

In [ ]:
# 将更新的项目表，和编码后的新增的项目表更新回总表
print("原全国商机表数据条数",len(opp_dp))
opp_totalnew = pd.read_excel(wpath+'全国新增的商机项目_加编.xlsx',encoding = "gbk")
opp_new = pd.concat([opp_dp,opp_renew,opp_totalnew])
print("新全国商机表数据条数",len(opp_new))
cou = opp_new['项目唯一编号\n（总部填写）'].value_counts()
print(cou[cou>1])
print("重复条数",len(cou[cou>1]))  # 重复条数与更新条数应该相同

In [ ]:
opp_new = opp_new.drop_duplicates(subset=['项目唯一编号\n（总部填写）'],keep='last', inplace=False)
print("去重后全国商机表数据条数",len(opp_new))
# 重新生成序号
opp_new['序号'] = np.arrange(1,len(opp_new))
print("更新后全国商用线下商机表数据条数",len(opp_new))
# 导出更新后的表格看下，其中“是否更新”列为“是”的是本次新增

opp_new.to_excel(wpath+'更新后全国商用线下商机表.xlsx',index=False,encoding='gbk')

In [ ]:
# 按区域拆分
# opp_new = pd.read_excel(wpath+'更新后全国商用线下商机表.xlsx',encoding='gbk')
opp_new['本周是否更新']=[np.nan]*len(opp_new) # 这句话是去掉“是否更新”一列的“是”
for name, group in opp_new.groupby('*大区'):
    print(name)
    group.to_excel(sep_path+name+"商用线下商机表.xlsx",encoding='gbk',index=False)